You can access the accompanying video here: 

<a href="https://blinks.bloomberg.com/screens/PLYR%20VOD%20361997308"> Bond Comparison BarPlot (12 min)</a>

In [1]:
import bql
import calendar
import bqviz

#instantiate bql service
bq = bql.Service()

In [2]:
# setup the data items
amt_issued_mthly = bq.func.sum(bq.func.group(bq.data.amt_issued(), [bq.func.month(bq.data.issue_dt()), bq.func.year(bq.data.issue_dt())]))

amt_prior_yr = bq.func.matches(amt_issued_mthly, amt_issued_mthly["'YEAR(ISSUE_DT())'"] < '2020')
amt_current_yr = bq.func.matches(amt_issued_mthly, amt_issued_mthly["'YEAR(ISSUE_DT())'"] == '2020')

avg_issuance = bq.func.avg(bq.func.ungroup(amt_prior_yr))
prior_monthly = bq.func.dropna(bq.func.groupsort(avg_issuance,sortby=avg_issuance["'MONTH(ISSUE_DT())'"],order='ASC'),remove_id=True)['value']
current_monthly = bq.func.dropna(bq.func.ungroup(amt_current_yr),remove_id=True)['value']

items = {'prior':prior_monthly,'current':current_monthly}

# construct the query universe
filter1 = bq.data.bics_level_2_industry_group_name() == 'Airlines'
filter2 = bq.data.issue_dt() > '2014-12-31'
univ = bq.univ.filter(bq.univ.debtuniv(types='ALL'), bq.func.and_(filter1, filter2))

# set additional params
params = {'mode':'cached','currency':'usd'}

# build request and execute
bql_request = bql.Request(univ, items, params)
bql_response = bq.execute(bql_request)

#extract data and format
data = bql.combined_df(bql_response)
data['month'] = [calendar.month_abbr[int(float(x))] for x in list(data.index)]
data.set_index('month',inplace=True)

# make viz and show
viz = bqviz.BarPlot(data[['prior','current']]).set_style()
viz.show()

GridBox(children=(Figure(animation_duration=500, axes=[Axis(color='white', grid_color='#3c3c3c', grid_lines='d…